In [1]:
import os
import glob


In [2]:
from dms2dec.dms_convert import dms2dec

In [3]:
import pandas as pd
import numpy as np
import re


In [4]:
city_row=pd.read_csv("C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_city.csv")
UrbanVilige_row=pd.read_csv("C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_UrbanVilige.csv")
vilige_row=pd.read_csv("C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/raw_data/raw_data_vilige.csv")
city_row['type_of_settlements'] = 'city'
UrbanVilige_row['type_of_settlements'] = 'Urban_Vilige'
vilige_row['type_of_settlements']='vilage'
row = pd.concat([city_row,UrbanVilige_row,vilige_row])


In [5]:
row.info()
print('Дублікатів: ',row.duplicated(subset=['longitude','longitude','city']).sum())
print('region: ' , len(row['region'].unique()))
row['region'].unique()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 27928 entries, 0 to 26267
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 27928 non-null  object
 1   year                 16446 non-null  object
 2   population           27792 non-null  object
 3   latitude             27884 non-null  object
 4   longitude            27884 non-null  object
 5   type_of_settlements  27928 non-null  object
 6   region               26268 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB
Дублікатів:  10
region:  26


array([nan, 'cherkasy', 'chernihiv', 'chernivtsi', 'dnipro', 'donetsk',
       'ivan_frank', 'kirivigrad', 'krym', 'kyiv', 'lugansk', 'lviv',
       'mykolayiv', 'odesa', 'poltava', 'rivne', 'sumsk', 'ternopil',
       'Vinitsa', 'Volins', 'xarkiv', 'xerson', 'xmelnitsk', 'zakarpat',
       'zaporiz', 'zitomer'], dtype=object)

In [14]:
data_row =  row.copy()
data_row= data_row[data_row['latitude'].notna()]
data_row= data_row[data_row['longitude'].notna()]
data_row['latitude_point']=data_row['latitude'].apply(dms2dec)
data_row['longitude_point']=data_row['longitude'].apply(dms2dec)
data_row
data_row.to_csv(r'C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/data/data_row.csv', index = True, header = True)

In [11]:
data =  row.copy()
data= data[data['latitude'].notna()]
data= data[data['longitude'].notna()]
data= data[data['year'].notna()]
data = data[data['year'] != '—']
data = data[data['year'] != '']
df=data
df = df.reindex(columns = ['city','region', 'type_of_settlements', 'year','year_new','year_century','year_','population','population_', 'latitude','longitude'])
df = df.reset_index(drop=True)
df.tail()


,city,region,type_of_settlements,year,year_new,year_century,year_,population,population_,latitude,longitude
16437,Ярунь (Новоград-Волинський район),zitomer,vilage,1540,NaN,NaN,NaN,3478,NaN,50°32′36″ пн. ш.,27°27′58″ сх. д.
16438,Ясна Поляна (Житомирський район),zitomer,vilage,1689,NaN,NaN,NaN,681,NaN,50°30′18″ пн. ш.,28°11′59″ сх. д.
16439,Ясопіль,zitomer,vilage,1585,NaN,NaN,NaN,20,NaN,49°52′37″ пн. ш.,28°06′29″ сх. д.
16440,Яструбенька,zitomer,vilage,18 століття,NaN,NaN,NaN,208,NaN,50°16′53″ пн. ш.,29°38′15″ сх. д.
16441,Яструбна (Житомирський район),zitomer,vilage,1611,NaN,NaN,NaN,47,NaN,50°17′37″ пн. ш.,29°39′37″ сх. д.


In [225]:
df['year_new']=''
df['year_century']=''

re_express = re.compile(r'(^.*)((до н.е.)|(до н. е.)|(до не\s))')
df['year_century'].loc[df['year'].str.contains(re_express)]='CE '
df['year'] = df['year'].replace(re_express, r'\1 ')

re_express = re.compile(r'(^.*)тис.')
df['year_century'].loc[df['year'].str.contains(re_express)]='millennium '
df['year'] = df['year'].replace(re_express, r'\1 ')


re_express = re.compile(r'(^.*)(стол){1}\w+')
df['year_new'].loc[df['year'].str.contains(re_express)]=df['year_new']+'century'
df['year'] = df['year'].replace(re_express, r'\1')

re_express = re.compile(r'(^.*)((стор){1}\w+)(.*)')
df['year_new'].loc[df['year'].str.contains(re_express)]=df['year_new']+'century'
df['year'] = df['year'].replace(re_express, r'\1')

re_express = re.compile(r'(^.*)ст(\.+|\s+)')
df['year_new'].loc[df['year'].str.contains(re_express)]=df['year_new']+'century'
df['year'] = df['year'].replace(re_express, r'\1 ')

re_express = re.compile(r'(^.*)ст$')
df['year_new'].loc[df['year'].str.contains(re_express)]=df['year_new']+'century'
df['year'] = df['year'].replace(re_express, r'\1 ')

re_express = re.compile(r'(^.*)([VXxIlХхX])(.*)([^(ст)])(.*)$')
df['year_new'].loc[df['year'].str.contains(re_express)]='century'



re_express = re.compile(r'^(^.*)(\d{4})(.*)')
df['year'] = df['year'].replace(re_express, r'\2')
re_express = re.compile(r'^(^.*)(\d{4})(\D+)(\w|\s|\d|\(|\)|\[|\]|\\|-)*')
df['year'] = df['year'].replace(re_express, r'\2')

re_express = re.compile('[а-яА-ФЦ-ЯЇїЄєі\'\_\.≈]')
df['year'] = df['year'].replace(re_express, '')

re_express = re.compile(r'^(^.*)(\d{3})(\D+)(\w|\s|\d|\(|\)|\[|\]|\\|-)*')
df['year'] = df['year'].replace(re_express, r'\2')


re_express = re.compile(r'^(^.*)(\d{2})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{4})(\D)*')
df['year'] = df['year'].replace(re_express, r'\5')

re_express = re.compile(r'^(^.*)(\d{2})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{3})(\D)*')
df['year'] = df['year'].replace(re_express, r'\2')

re_express = re.compile(r'^(^.*)(\d{2})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{2})(\D)*')
df['year'] = df['year'].replace(re_express, r'\2')

re_express = re.compile(r'^(^.*)(\d{1})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{4})(\D)*')
df['year'] = df['year'].replace(re_express, r'\5')

re_express = re.compile(r'^(^.*)(\d{1})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{3})(\D)*')
df['year'] = df['year'].replace(re_express, r'\5')

re_express = re.compile(r'^(^.*)(\d{1})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{2})(\D)*')
df['year'] = df['year'].replace(re_express, r'\5')

re_express = re.compile(r'^(\s|\(|\)|\[|\]|\\|-|–)(\d{1})(\D+)(\w|\s|\(|\)|\[|\]|\\|-|–)*(\d{2})(\D)*')
df['year'] = df['year'].replace(re_express, r'\2')



re_express = re.compile(r'\d{3,6}')
#re_express = re.compile(r'(^C{1}^E{1})')
df['year_new'].loc[df['year'].str.contains(re_express)]=np.nan

#df['year_new'] = np.where((df.year_century == 'CE '),df.year_century, np.nan )
#df['year_new'] = np.where((df.year_century == 'millennium'),df.year_century, np.nan )
df['year_new'] = (np.where((df['year_new'].isna()==True),np.nan, df['year_new'] ))
df['year_new'] = np.where((df.year_century == 'CE '), (np.where((df.year_new.isna()==True),df.year_century, df.year_century+ df.year_new )), (np.where((df.year_new.isna()==True),np.nan, df.year_new )) )

df['year_century'] = np.nan 

df[df['city'] =='Потапці (село)']



C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,city,region,type_of_settlements,year,year_new,year_century,year_,population,population_,latitude,longitude
1500,Потапці (село),cherkasy,vilage,1777,NaN,NaN,NaN,309 осіб (,NaN,49°50′29″ пн. ш.,31°14′22″ сх. д.


In [226]:

re_express = re.compile('(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='1'

re_express = re.compile('(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='2'
re_express = re.compile('(I|І|l)(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='3'
re_express = re.compile('(I|І|l)V')
df['year_century'].loc[df['year'].str.contains(re_express)]='4'
re_express = re.compile('V')
df['year_century'].loc[df['year'].str.contains(re_express)]='5'
re_express = re.compile('V(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='6'
re_express = re.compile('V(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='7'
re_express = re.compile('V(I|І|l)(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='8'
re_express = re.compile('(I|І|l)(X|Х|X)')
df['year_century'].loc[df['year'].str.contains(re_express)]='9'

re_express = re.compile('(X|Х|X)')
df['year_century'].loc[df['year'].str.contains(re_express)]='10'

re_express = re.compile('(X|Х|X)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='11'
re_express = re.compile('(X|Х|X)(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='12'
re_express = re.compile('(X|Х|X)(I|І|l)(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='13'
re_express = re.compile('(X|Х|X)(I|І|l)V')
df['year_century'].loc[df['year'].str.contains(re_express)]='14'
re_express = re.compile('(X|Х|X)V')
df['year_century'].loc[df['year'].str.contains(re_express)]='15'
re_express = re.compile('(X|Х|X)V(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='16'
re_express = re.compile('(X|Х|X)V(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='17'
re_express = re.compile('(X|Х|X)V(I|І|l)(I|І|l)(I|І|l)')
df['year_century'].loc[df['year'].str.contains(re_express)]='18'
re_express = re.compile('(X|Х|X)(I|І|l)(X|Х|X)')
df['year_century'].loc[df['year'].str.contains(re_express)]='19'
re_express = re.compile('(X|Х|X)(X|Х|X)')
df['year_century'].loc[df['year'].str.contains(re_express)]='20'



C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [227]:
#df['year_'].loc[(df['year_new'] == 'century')]=  df.year + '---'
df['year_'] = np.where((df.year_new == 'century'),df.year, np.nan )
df['year_'] =  np.where((df.year_new == 'century'),np.where((df['year_century'].isna()==True),df.year +'00' , df.year_century +'00' ), np.nan )
df['year_'] =  np.where((df.year_new == 'millennium'),np.where((df['year_century'].isna()==True),df.year +'000' , df.year_century +'000' ), df['year_']  )
df['year_'] =  np.where((df.year_new == 'CE century'),np.where((df['year_century'].isna()==True), '-' + df.year +'00' , '-' +  df.year_century +'00' ), df['year_'] )
df['year_'] =  np.where((df.year_new == 'CE millennium'),np.where((df['year_century'].isna()==True),  '-' +df.year +'000' , '-' + df.year_century +'000' ), df['year_']  )
df['year_'] =  np.where((df.year_new == 'CE '),np.where((df['year_century'].isna()==True), '-' + df.year  , '-' +  df.year_century  ), df['year_'] )

df['year_'] = df['year_'].replace(re.compile(r'\s'), r'')


df['year_'] = np.where((df['year_'].isna()==True),df['year'], df['year_'] )

df[['city','region', 'type_of_settlements','year_','population','latitude','longitude']]




,city,region,type_of_settlements,year_,population,latitude,longitude
0,Авдіївка,NaN,city,1778,32 843 (1.1.2019),48°08′05″ пн. ш.,37°44′40″ сх. д.
1,Аджи-Мушкай,NaN,city,1666,?,45°23′10″ пн. ш.,36°31′15″ сх. д.
2,Алмазна,NaN,city,1870,4242 (01.01.2017),48°31′18″ пн. ш.,38°34′46″ сх. д.
3,Алупка,NaN,city,1000,8528,44°25′18″ пн. ш.,34°02′50″ сх. д.
4,Алушта,NaN,city,500,28 295,44°40′09″ пн. ш.,34°24′11″ сх. д.
...,...,...,...,...,...,...,...
16437,Ярунь (Новоград-Волинський район),zitomer,vilage,1540,3478,50°32′36″ пн. ш.,27°27′58″ сх. д.
16438,Ясна Поляна (Житомирський район),zitomer,vilage,1689,681,50°30′18″ пн. ш.,28°11′59″ сх. д.
16439,Ясопіль,zitomer,vilage,1585,20,49°52′37″ пн. ш.,28°06′29″ сх. д.
16440,Яструбенька,zitomer,vilage,1800,208,50°16′53″ пн. ш.,29°38′15″ сх. д.


In [228]:
re_express = re.compile('(^\D*)(.*)(\({1}\d{1,2}(\.|\s){1}\d{1,2}(\.|\s){1}\d{2,4}\){1})')
df['population'] = df['population'].replace(re_express, r'\2 ')
re_express = re.compile('(^\D*)(.*)(\({1}\d{1,2}(\.|\s){1}\d{1,2}(\.|\s){1}\d{2,4}\s*)$')
df['population'] = df['population'].replace(re_express, r'\2 ')
re_express = re.compile('(^\D*)(.*)(\({1}\d{4}\){1})$')
df['population'] = df['population'].replace(re_express, r'\2 ')
re_express = re.compile('(^\D*)(.*)(\(\s*)$')
df['population'] = df['population'].replace(re_express, r'\2 ')
re_express = re.compile('(^\D*)((\d|\s)*)(.*)$')
df['population'] = df['population'].replace(re_express, r'\2 ')

df['population'] = df['population'].replace(re.compile(r'\s'), r'')



In [229]:
df['latitude_point']=df['latitude'].apply(dms2dec)
df['longitude_point']=df['longitude'].apply(dms2dec)
df


,city,region,type_of_settlements,year,year_new,year_century,year_,population,population_,latitude,longitude,latitude_point,longitude_point
0,Авдіївка,NaN,city,1778,NaN,NaN,1778,32843,NaN,48°08′05″ пн. ш.,37°44′40″ сх. д.,48.134722,37.744444
1,Аджи-Мушкай,NaN,city,1666,NaN,NaN,1666,,NaN,45°23′10″ пн. ш.,36°31′15″ сх. д.,45.386111,36.520833
2,Алмазна,NaN,city,1870,NaN,NaN,1870,4242,NaN,48°31′18″ пн. ш.,38°34′46″ сх. д.,48.521667,38.579444
3,Алупка,NaN,city,Х,century,10,1000,8528,NaN,44°25′18″ пн. ш.,34°02′50″ сх. д.,44.421667,34.047222
4,Алушта,NaN,city,V,century,5,500,28295,NaN,44°40′09″ пн. ш.,34°24′11″ сх. д.,44.669167,34.403056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16437,Ярунь (Новоград-Волинський район),zitomer,vilage,1540,NaN,NaN,1540,3478,NaN,50°32′36″ пн. ш.,27°27′58″ сх. д.,50.543333,27.466111
16438,Ясна Поляна (Житомирський район),zitomer,vilage,1689,NaN,NaN,1689,681,NaN,50°30′18″ пн. ш.,28°11′59″ сх. д.,50.505000,28.199722
16439,Ясопіль,zitomer,vilage,1585,NaN,NaN,1585,20,NaN,49°52′37″ пн. ш.,28°06′29″ сх. д.,49.876944,28.108056
16440,Яструбенька,zitomer,vilage,18,century,NaN,1800,208,NaN,50°16′53″ пн. ш.,29°38′15″ сх. д.,50.281389,29.637500


In [230]:
#df.loc[df['population'] == some_value]
df.population = df['population'].apply(pd.to_numeric, errors='coerce') 
df.year_ = df['year_'].apply(pd.to_numeric, errors='coerce') 

print('Пусті: ',len(df[df['year_'].isna()])  )
df= df[df['year_'].notna()]


Пусті:  98


In [239]:
data_new =  df[['city','region', 'type_of_settlements', 'year_','population','latitude_point','longitude_point']].copy()
data_new = data_new.rename(columns={'year_': 'year'})

data_new.loc[data_new['city'] == 'Шабельники (Золотоніський район)', 'year'] = 1830

data_new=data_new.sort_values(by='year', ascending=True)
data_new = data_new.reset_index(drop=True)

#city_new= city_new[city_new['population'].notna()]
data_new




,city,region,type_of_settlements,year,population,latitude_point,longitude_point
0,Руське Поле,zakarpat,vilage,-1303.0,6112.0,48.056389,23.538333
1,Жулин,lviv,vilage,-1240.0,968.0,49.177222,23.773056
2,Кілія,NaN,city,-862.0,NaN,45.467500,29.237778
3,Феодосія,NaN,city,-600.0,69145.0,45.048889,35.379167
4,Керч,NaN,city,-530.0,149566.0,45.350000,36.466667
...,...,...,...,...,...,...,...
16339,Мірошниківка (Херсонський район),xerson,vilage,2008.0,NaN,46.768889,32.582778
16340,Калинник,lugansk,vilage,2009.0,86.0,48.114444,39.807778
16341,Подолянці,zitomer,vilage,2009.0,390.0,50.019722,28.101667
16342,Мазурівка (Чортківський район),ternopil,vilage,2015.0,164.0,49.049444,25.681389


In [240]:
data_new.to_csv(r'C:/Users/Administrator/Desktop/робота_сортування по даті/жовтень/DF/data/data.csv', index = True, header = True)